In [1]:
# !pip install tensorflow pandas numpy matplotlib scikit-learn joblib pydot==2.0.0 setuptools

In [2]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import re
import joblib

In [3]:
# Define the maximum limits (same as used during training)
MAX_X = 220  # Max X-axis limit
MAX_Y = 220  # Max Y-axis limit
MAX_Z = 250  # Max Z-axis limit
MAX_TEMP = 260  # Safe extruder temp limit
MAX_BED_TEMP = 120  # Safe bed temp limit
MAX_SPEED = 5000  # Safe movement speed limit

In [4]:
# Load the saved model (replace 'gcode_model.h5' with your model's filename)
model = tf.keras.models.load_model(r'C:\Users\awebb\Documents\Programming\Work\SIIL\Cyber Fair Malicious Gcode\gcode_malicious_detection_model.keras')

In [5]:
# Helper function to extract features from a single G-code file (same as used in training)
def extract_gcode_features(file_path):
    features = {
        'temp_extruder': 0,
        'temp_bed': 0,
        'max_x': 0,
        'max_y': 0,
        'max_z': 0,
        'max_speed': 0,
        'max_extrusion': 0,
        'fan_speed': 0,
        'num_commands': 0,
        'out_of_bounds_moves': 0
    }

    with open(file_path, 'r') as f:
        for line in f:
            features['num_commands'] += 1

            # Extract temperature commands
            if line.startswith('M104') or line.startswith('M109'):
                temp = extract_value_from_line(line, 'S')
                if temp:
                    features['temp_extruder'] = max(features['temp_extruder'], float(temp))
            
            if line.startswith('M140') or line.startswith('M190'):
                temp = extract_value_from_line(line, 'S')
                if temp:
                    features['temp_bed'] = max(features['temp_bed'], float(temp))
            
            # Extract movement commands
            if line.startswith('G0') or line.startswith('G1'):
                x = extract_value_from_line(line, 'X')
                y = extract_value_from_line(line, 'Y')
                z = extract_value_from_line(line, 'Z')
                f = extract_value_from_line(line, 'F')
                e = extract_value_from_line(line, 'E')

                if x: features['max_x'] = max(features['max_x'], float(x))
                if y: features['max_y'] = max(features['max_y'], float(y))
                if z: features['max_z'] = max(features['max_z'], float(z))
                if f: features['max_speed'] = max(features['max_speed'], float(f))
                if e: features['max_extrusion'] = max(features['max_extrusion'], float(e))

                # Check for out-of-bounds movements
                if (x and float(x) > MAX_X) or (y and float(y) > MAX_Y) or (z and float(z) > MAX_Z):
                    features['out_of_bounds_moves'] += 1
            
            # Extract fan speed
            if line.startswith('M106'):
                fan_speed = extract_value_from_line(line, 'S')
                if fan_speed:
                    features['fan_speed'] = max(features['fan_speed'], float(fan_speed))

    return features

In [6]:
# Helper function to extract a parameter value (e.g., X, Y, Z, S) from a G-code line
def extract_value_from_line(line, param):
    match = re.search(rf"{param}([-\d.]+)", line)
    if match:
        return match.group(1)
    return None

In [7]:
# Normalize features based on the scaler used during training
def normalize_features(features, scaler):
    feature_values = np.array(list(features.values())).reshape(1, -1)  # Convert dict to 2D array
    return scaler.transform(feature_values)

In [ ]:
# Load the pre-trained scaler (assumes scaler was saved during training as 'scaler.pkl')
scaler = joblib.load(r'C:\Users\awebb\Documents\Programming\Work\SIIL\Cyber Fair Malicious Gcode\scaler.pkl')

In [9]:
# Test the model on one good G-code file and one bad G-code file
good_gcode_file = r'C:\Users\awebb\Documents\Programming\Work\SIIL\Cyber Fair Malicious Gcode\demo\cube.gcode'  # Path to your good G-code file
bad_gcode_file = r'C:\Users\awebb\Documents\Programming\Work\SIIL\Cyber Fair Malicious Gcode\demo\malicious_cube.gcode'    # Path to your bad G-code file

In [10]:
# Extract features from the good and bad G-code files
good_features = extract_gcode_features(good_gcode_file)
bad_features = extract_gcode_features(bad_gcode_file)

In [ ]:
# Normalize the features
good_features_normalized = normalize_features(good_features, scaler)
bad_features_normalized = normalize_features(bad_features, scaler)

In [12]:
# Make predictions
good_prediction = model.predict(good_features_normalized)
bad_prediction = model.predict(bad_features_normalized)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


In [13]:
# Convert predictions to binary labels (0 for good, 1 for malicious)
good_label = (good_prediction > 0.5).astype("int32")
bad_label = (bad_prediction > 0.5).astype("int32")

In [14]:
# Output the predictions
print(f"Good G-code prediction: {good_prediction[0][0]:.4f} (label: {good_label[0][0]})")
print(f"Bad G-code prediction: {bad_prediction[0][0]:.4f} (label: {bad_label[0][0]})")

Good G-code prediction: 0.0001 (label: 0)
Bad G-code prediction: 1.0000 (label: 1)


In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,333 (138.02 KB)

 Trainable params: 11,777 (46.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 23,556 (92.02 KB)